In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from numpy import array
from numpy import hstack
import re
import pickle
from math import sqrt
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error

In [39]:
train_sales = pd.read_csv('./data/sales_train_validation.csv')
sell_prices = pd.read_csv('./data/sell_prices.csv')
calendar = pd.read_csv('./data/calendar.csv')
submission_file = pd.read_csv('./data/sample_submission.csv')

In [40]:
eventsArray = (calendar['event_name_1'].append(calendar['event_name_2'])).unique()
eventTypesArray = (calendar['event_type_1'].append(calendar['event_type_2'])).unique()
weekDaysArray = calendar['weekday'].unique()
states = train_sales['state_id'].unique()

In [41]:
dayIndexString = 'd_'


In [42]:
def getLastDayOfTrainset():
    line2 = train_sales.columns[-1]
    temp1 = re.findall(r'\d+', line2) # through regular expression
    return list(map(int, temp1))[0]


In [43]:
def date_to_nth_day(date, format='%Y%m%d'):
    date = pd.to_datetime(date, format=format)
    new_year_day = pd.Timestamp(year=date.year, month=1, day=1)
    return (date - new_year_day).days + 1

def eventsToDaysBefore(day_id, nmbr_days_in_future = 30):
    events = np.full(eventsArray.size, nmbr_days_in_future)
    events_types = np.full(eventTypesArray.size, nmbr_days_in_future)
    nmbr_days_in_future = 30
    for i in range(0, nmbr_days_in_future):
        ## get day in future
        idx = calendar.index[calendar['d'] == dayIndexString + str(day_id + i)]
        dayDetails = calendar.loc[idx]
        event_1 = dayDetails['event_name_1'].iloc[0]
        # Process first event
        if(not pd.isnull(event_1)):
            index = np.where(eventsArray==event_1)
            events[index] = i
        event_2 = dayDetails['event_name_2'].iloc[0]

        # Process second event
        if(not pd.isnull(event_2)):
            index = np.where(eventsArray==event_2)
            events[index] = i

        # Process first event type
        event_t_1 = dayDetails['event_type_1'].iloc[0]
        if(not pd.isnull(event_t_1)):
            index = np.where(eventTypesArray==event_t_1)
            events_types[index] = i

        # Process second event type
        event_t_2 = dayDetails['event_type_2'].iloc[0]
        if(not pd.isnull(dayDetails['event_type_2'].iloc[0])):
            index = np.where(eventTypesArray==event_t_2)
            events_types[index] = i
    return events, events_types
    
def getCalendarInfo(day_id):
    idx = calendar.index[calendar['d'] == day_id]
    dayDetails = calendar.loc[idx]
    dayDetails['date'] =  date_to_nth_day(dayDetails['date'].iloc[0], '%Y-%m-%d')
    # Remove weekday because table included wday value
    dayDetails.drop(columns=['weekday'], inplace = True)
    # Remove week id because I don't know what's the added value because we have year and number of day in year
    dayDetails.drop(columns=['wm_yr_wk'], inplace = True)
    
    dayDetails.drop(columns=['event_name_1'], inplace = True)
    dayDetails.drop(columns=['event_name_2'], inplace = True)
    dayDetails.drop(columns=['event_type_1'], inplace = True)
    dayDetails.drop(columns=['event_type_2'], inplace = True)
    
    # Reset year of 2011 to index year 0
    dayDetails['year'] = dayDetails['year'] - 2011  
    dayDetails['d'] = int(dayDetails['d'].str[2:])
    return dayDetails
    


In [44]:
def trainBatch(day_id = 1, day_range = 28):
    train_container_panda = pd.DataFrame(index=train_sales.index)

    for i in range(0, day_range):
            # append sales of day
            sales = train_sales[dayIndexString + str(day_id + i)]
            train_container_panda['sale_' + str(i)] = sales
    for i in range(0, 1):
            # Get basic calendar info
            calendarInfo = getCalendarInfo('d_' + str(day_id + day_range + i)).values[0]
            # Get one-hot occur event on this day od
            eventInfo = eventsToDaysBefore(day_id + day_range + i)
            # Concatenate calendar info and event info
            calendarInfo = np.concatenate((calendarInfo, eventInfo[0]))
            # Concatenate info with event type info
            calendarInfo = np.concatenate((calendarInfo, eventInfo[1]))
            # append the event based information 
            columnIndex = 0
            for col in calendarInfo:
                train_container_panda['cal_' + str(day_id + day_range + i) + '_' + str(columnIndex)] = col
                columnIndex = columnIndex + 1
    return train_container_panda

def testBatch(day_id = 1, day_range = 28):
    train_container_panda = pd.DataFrame(index=train_sales.index)

    for i in range(0, day_range):
            # append sales of d|ay
            print(dayIndexString + str(day_id + i))
            sales = train_sales[dayIndexString + str(day_id + i)]
            train_container_panda['sale_' + str(day_id + i)] = sales
               
    return train_container_panda

def getDataset(numberOfDays):
    lastTrainDay = getLastDayOfTrainset() + 1
    
    startIndex = lastTrainDay - numberOfDays;
    return trainBatch(startIndex, numberOfDays).to_numpy()

def getTestDataset(numberOfDays):
    lastTrainDay = getLastDayOfTrainset() + 1
    
    startIndex = lastTrainDay - numberOfDays;
    return testBatch(startIndex, numberOfDays)

In [45]:
with open('voting_batches.test', 'rb') as fid:
    regr = pickle.load(fid)
    


In [46]:
def save_sumission_file(results, submission):
    submision_data =  pd.DataFrame(submission_file['id'])

    for col_index in range (1, len(results[0])+1):
        submision_data['F' + str(col_index)] = 0
    pred_data = pd.DataFrame(results)
    pred_data.columns = submision_data.columns[1:]

    submision_data.loc[pred_data.index] = pred_data
    submision_data['id'] = submission_file['id']
    submision_data.head()
    submision_data.to_csv('final_submission_batch_all_lin.csv', index = False)

In [47]:
test_dataset = getDataset(56)
prediction_results = regr.predict(test_dataset)


In [48]:
save_sumission_file(prediction_results, submission_file)

In [49]:
original_results = getTestDataset(28)


d_1886
d_1887
d_1888
d_1889
d_1890
d_1891
d_1892
d_1893
d_1894
d_1895
d_1896
d_1897
d_1898
d_1899
d_1900
d_1901
d_1902
d_1903
d_1904
d_1905
d_1906
d_1907
d_1908
d_1909
d_1910
d_1911
d_1912
d_1913


In [50]:
print(test_dataset)



[[ 0  2  0 ... 13 30  6]
 [ 0  0  0 ... 13 30  6]
 [ 0  0  0 ... 13 30  6]
 ...
 [ 1  2  2 ... 13 30  6]
 [ 2  2  2 ... 13 30  6]
 [ 2  0  5 ... 13 30  6]]


In [51]:
sqrt(mean_squared_error(original_results, prediction_results))

1.8580054091231908

In [52]:
sqrt(mean_squared_error(original_results, prediction_results))

1.8580054091231908

In [53]:
mean_squared_error(np.ones(np.shape(original_results)), prediction_results)

8.553889477615808

In [22]:
test_dataset = getDataset(28)

In [ ]:
## RMSE 1.927 for batches of 56 of 10 estimators
## RMSE 1.99 for batches of 56 of 1 estimators
## RMSE 1.91 for retrain for each batch of 56 
## RMSE 1.927 for batches of 56 of 10 estimators